# Build out a neural net library

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# plot graph
import graphviz

def trace(root):
    nodes, edges = set(), set()
    def build(root):
        if root not in nodes:
            nodes.add(root)
        for v in root._children:
            edges.add((v, root))
            build(v)
    build(root)
    return nodes, edges

def draw_graph(root):
    f = graphviz.Digraph(format='svg', graph_attr={'rankdir':"LR"})
    nodes, edges = trace(root)
    for v in nodes:
        f.node(str(id(v)), label="{%s|data %.4f|grad %.4f}" %(v.label, v.data, v.grad) , shape='record')
        if v._op:
            f.node(f'{str(id(v))}_{v._op}', label=f"{v._op}")
            f.edge(f'{str(id(v))}_{v._op}', str(id(v)))
    for v1, v2 in edges:
        f.edge(str(id(v1)), f'{str(id(v2))}_{v2._op}')
    return f

In [80]:
import math

class Value():
    def __init__(self, data, label='', _children=[], _op="", grad = 0):
        self.data = data
        self.label = label
        self._children = set(_children)
        self._op = _op
        self.grad = grad
        self._backward = lambda:None
    def __repr__(self):
        return f"Value({self.label}={self.data})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, _children=[self, other], _op="+")
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
    
    def __sub__(self, other):
        return self + (-1) * other
    
    def __rsub__(self, other):
        return (-1) * self + other
    
    def __radd__(self, other):
        return self + other

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, _children=[self, other], _op="*")
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __rmul__(self, other):
        return self * other

    def tanh(self):
        tanh = math.tanh(self.data)
        out = Value(tanh, _children=[self, ], _op="tanh")
        def _backward():
            self.grad += (1 - out.data**2) * out.grad
        out._backward = _backward
        return out
    
    def exp(self):
        out = Value(math.exp(self.data), _children=[self, ], _op='exp')
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out

    def __pow__(self, n):
        out = Value(self.data**n, _children=[self, ], _op='pow')
        def _backward():
            self.grad += n * self.data**(n-1) * out.grad
        out._backward = _backward
        return out
    
    def __truediv__(self, other):
        return self * other**(-1)

    def backward(self):
        self.grad = 1.0
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for o in v._children:
                    build_topo(o)
                topo.append(v)

        build_topo(self)
        for node in reversed(topo):
            node._backward()

In [198]:
import random

class Neuron():
    def __init__(self, n_in):
        self.weights = [Value(random.uniform(-1, 1), f'w_{i}') for i in range(n_in)]
        self.b = Value(random.uniform(-1, 1), label='b')
        
    def __call__(self, xs):
        return sum([w*x for w,x in zip(self.weights, xs)], self.b)
    
    def parameters(self):
        return [self.b] + self.weights

class Layer():
    def __init__(self, n_in, n_out):
        self.neurons = [Neuron(n_in) for _ in range(n_out)]
        
    def __call__(self, x):
        return [neuron(x) for neuron in self.neurons]
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP():
    def __init__(self, n_int, n_outs):
        sizes = [n_int] + n_outs
        self.layers = [Layer(sizes[i], sizes[i+1]) for i in range(len(n_outs))]

    def __call__(self, x):
        for l in self.layers:
            x = l(x)
        x = [o.tanh() for o in x]
        return x if len(x) != 1 else x[0]
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

net = MLP(3, [4, 4, 1])

In [199]:
xs = [
        [1.0, 2.0, 3.0],
        [-1.0, -2.0, 3.0],
        [2.0, -1.0, 1.0],
        [-1.0, 0.0, 0.0],
     ]
ys = [1, -1, 1, -1]

In [222]:
# backward
# zero grad
for _ in range(10):
    y_pred = [net(x) for x in xs]
    print(y_pred)
    L = sum([(ygt - yout)**2 for ygt, yout in zip(ys, y_pred)]); L

    for p in net.parameters():
        p.grad = 0
    L.backward()

    # update param
    for p in net.parameters():
        p.data += -0.01 * p.grad

[Value(=0.9999979646930708), Value(=-0.9701802042009974), Value(=0.96360191340795), Value(=-0.9880742220310944)]
[Value(=0.9999979724521499), Value(=-0.970264692785905), Value(=0.9637047944630939), Value(=-0.9881000361957057)]
[Value(=0.9999979801442545), Value(=-0.9703485378880123), Value(=0.9638068911520428), Value(=-0.9881256709397696)]
[Value(=0.9999979877702769), Value(=-0.9704317474186374), Value(=0.9639082131313246), Value(=-0.9881511283555617)]
[Value(=0.9999979953310941), Value(=-0.9705143291553467), Value(=0.9640087698939797), Value(=-0.9881764105012469)]
[Value(=0.9999980028275669), Value(=-0.9705962907448278), Value(=0.9641085707730743), Value(=-0.9882015194015914)]
[Value(=0.9999980102605411), Value(=-0.9706776397056831), Value(=0.9642076249451234), Value(=-0.9882264570486571)]
[Value(=0.9999980176308474), Value(=-0.970758383431155), Value(=0.9643059414334241), Value(=-0.9882512254024785)]
[Value(=0.9999980249393018), Value(=-0.9708385291917795), Value(=0.9644035291113039)

In [158]:
net.parameters()[0].grad

0.8958342589007484

In [85]:
# draw_graph(out)

### Convert this file to md

In [5]:
from IPython.core.display import Javascript

In [6]:
%%js
IPython.notebook.kernel.execute('this_notebook = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [7]:
this_notebook

NameError: name 'this_notebook' is not defined

In [ ]:
!jupyter nbconvert --to markdown {this_notebook} --output-dir=../_posts